In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as e:
    credential = InteractiveBrowserCredential()

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Please run 'az login' to set up an account
	AzurePowerShellCredential: Az.Account module >= 2.2.0 is not installed
	AzureDeveloperCliCredential: {"type":"consoleMessage","timestamp":"2024-01-05T09:44:40.2933487-05:00","data":{"message":"fetching token: reauthentication required, run `azd auth login --scope https://management.azure.com/.default` to log in\n"}}

To mitigate this issue, please refer to the t

In [ ]:
from azure.ai.ml import MLClient

ml_client = MLClient.from_config(credential=credential)

Deploy to online endpoint

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M")

endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for model deployment",
    auth_mode="key"
)

In [ ]:
ml_client.begin_create_or_update(endpoint).result()

In [ ]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

model = Model(
    path="./mlflow-model",
    type=AssetTypes.MLFLOW_MODEL,
    description="diabetes mlflow model"
)

In [ ]:
deployment_name = "diabetes-model-deployment"

deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

In [ ]:
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment_name,
    request_file="sample-data.json"
)

print(response)